# Sentiment Analysis For Turkish
[Data Source](https://github.com/fthbrmnby/turkish-text-data)


In [1]:
import pandas as pd
negative_data = pd.read_csv('reviews.neg', delimiter='\t')
positive_data = pd.read_csv('reviews.pos', delimiter='\t')

# Changing col names for concatting
negative_data['label'] = 'neg'
negative_data.rename(columns={'beklentimin altında bir ürün kaliteli değil': 'review'}, inplace=True)

positive_data['label'] = 'pos'
positive_data.rename(columns={'fena değil paraya göre iyi.': 'review'}, inplace=True)
# deleting big amount of rows for balancing data frames.
positive_data = positive_data.drop(index=range(0, 204000), axis=0)
df = pd.concat([positive_data, negative_data])
positive_data

,review,label
204000,"zamanında teslimat yapıldı, ayrıca ürün kalite...",pos
204001,gerçekten harika bir ürün kargo da hemen 2 gün...,pos
204002,fiyatına gore gayet güzel bir avize. taşları d...,pos
204003,ürün mükemmel salona muhteşem oldu. şık bir ür...,pos
204004,ürün beklediğim süreden daha kısa sürede elime...,pos
...,...,...
219738,ürün çok hızlı ve hediye paketi içerisinde gel...,pos
219739,ürün yanında göndermiş olduğunuz hediye çok in...,pos
219740,hepsi burada tarihinde ilk defa yorum yapıyoru...,pos
219741,çok güzel bir ürün mağazaya ve hepsiburadaya ç...,pos


In [2]:
# Shuffling data.
df = df.sample(frac=1).reset_index(drop=True)
df.head()
df['label'].value_counts()

pos    15743
neg    14878
Name: label, dtype: int64

## Preprocessing

In [3]:
import string
import re
import nltk
from nltk.corpus import stopwords

punctuations = string.punctuation
ineffective_words = stopwords.words('turkish')
print(punctuations)
print(ineffective_words)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [4]:
# Transforming lowercase to words.
df['review'].str.lower()

0        basınçlı yıkama makinesı, şampuan ve bu eldive...
1        dünya edebiyatında fantastik,korku,cinsellik k...
2        renkleri çok da canlı değil ve resim flu  ,kar...
3        bende sipariş verdm umarım iyi çıkar.ve erkend...
4        barlar ın üstünde sünger yok çok sert plastik ...
                               ...                        
30616    kokusuz olduğu için almıştım. 4-5 kere farklı ...
30617    anneme hediye olarak almıştım ama çekiş gücü ç...
30618    telefonu aldıktan 1 ay sonra bir tane de eşime...
30619    şarj ömrü çook ama çok az. şarjdayken bile kul...
30620    koku yapıyor ısıtmıyor ucuz diye almaya gerek ...
Name: review, Length: 30621, dtype: object

Shows deleting numbers and ineffective words

In [5]:
for d in df['review'].head():

    print(d+ '\n----------------------------------')
    # Removing ineffective words.
    temp = ''
    for word in d.split():
        if word not in ineffective_words and not word.isnumeric():
            temp += word + ' '
    print(temp + '\n******************************')


basınçlı yıkama makinesı, şampuan ve bu eldiveni hepsiburadadan aldım. eldiven çok bol ve sürekli elden çıkıyor etrafındaki püsküller de çok az pek işe yaradığını söyleyemem. eski usul fırça kullanırsanız işiniz daha kolaylaşır.
----------------------------------
basınçlı yıkama makinesı, şampuan eldiveni hepsiburadadan aldım. eldiven bol sürekli elden çıkıyor etrafındaki püsküller pek işe yaradığını söyleyemem. eski usul fırça kullanırsanız işiniz kolaylaşır. 
******************************
dünya edebiyatında fantastik,korku,cinsellik konulu romanların öne çıktığı bir zamanda konusu ve anlatımıyla farklı duyguları içinde barındıran samimi okunası bir kitap.sonunda size kazandırdıkları ve hatırlattıklarıyla övgüye değer bir hikaye .
----------------------------------
dünya edebiyatında fantastik,korku,cinsellik konulu romanların öne çıktığı bir zamanda konusu anlatımıyla farklı duyguları içinde barındıran samimi okunası bir kitap.sonunda size kazandırdıkları hatırlattıklarıyla övgüye d

Shows deleting punctuations

In [6]:
for d in df['review'].head():
    print(d + '\n----------------------------')
    temp = ''
    for word in d:
        if word not in punctuations:
            temp += word
    print(temp + '\n*************************')
    d = temp

basınçlı yıkama makinesı, şampuan ve bu eldiveni hepsiburadadan aldım. eldiven çok bol ve sürekli elden çıkıyor etrafındaki püsküller de çok az pek işe yaradığını söyleyemem. eski usul fırça kullanırsanız işiniz daha kolaylaşır.
----------------------------
basınçlı yıkama makinesı şampuan ve bu eldiveni hepsiburadadan aldım eldiven çok bol ve sürekli elden çıkıyor etrafındaki püsküller de çok az pek işe yaradığını söyleyemem eski usul fırça kullanırsanız işiniz daha kolaylaşır
*************************
dünya edebiyatında fantastik,korku,cinsellik konulu romanların öne çıktığı bir zamanda konusu ve anlatımıyla farklı duyguları içinde barındıran samimi okunası bir kitap.sonunda size kazandırdıkları ve hatırlattıklarıyla övgüye değer bir hikaye .
----------------------------
dünya edebiyatında fantastikkorkucinsellik konulu romanların öne çıktığı bir zamanda konusu ve anlatımıyla farklı duyguları içinde barındıran samimi okunası bir kitapsonunda size kazandırdıkları ve hatırlattıklarıyla

In [7]:
# Deleting Punctuations
df['review'] = df['review'].str.replace('[^\w\s]','')
#df['review'].apply(lambda x: [item for item in x if item not in ineffective_words])
df.head(5)

C:\Users\tyyp-\AppData\Local\Temp/ipykernel_15016/1973213164.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review'] = df['review'].str.replace('[^\w\s]','')


,review,label
0,basınçlı yıkama makinesı şampuan ve bu eldiven...,neg
1,dünya edebiyatında fantastikkorkucinsellik kon...,pos
2,renkleri çok da canlı değil ve resim flu karg...,neg
3,bende sipariş verdm umarım iyi çıkarve erkende...,neg
4,barlar ın üstünde sünger yok çok sert plastik ...,neg


In [8]:
# Deleting stop words-ineffective words
df['review'] = df['review'].apply(lambda x: ' '.join(x for x in x.split() if x not in ineffective_words))

df.head(5)

,review,label
0,basınçlı yıkama makinesı şampuan eldiveni heps...,neg
1,dünya edebiyatında fantastikkorkucinsellik kon...,pos
2,renkleri canlı değil resim flu kargo 4 günde e...,neg
3,bende sipariş verdm umarım iyi çıkarve erkende...,neg
4,barlar ın üstünde sünger yok sert plastik yumu...,neg


Save cleaned data

In [9]:
df.to_csv(r'./cleaned.csv', index=False)

import cleaned data

In [11]:
import pandas as pd
df = pd.read_csv('cleaned.csv', sep=",", names=['review', 'label'])
# Deleting first row(header)
df = df.iloc[1 : ]
df.head(5)

,review,label
1,basınçlı yıkama makinesı şampuan eldiveni heps...,neg
2,dünya edebiyatında fantastikkorkucinsellik kon...,pos
3,renkleri canlı değil resim flu kargo 4 günde e...,neg
4,bende sipariş verdm umarım iyi çıkarve erkende...,neg
5,barlar ın üstünde sünger yok sert plastik yumu...,neg


## Let's train model

In [12]:
from sklearn.model_selection import train_test_split
# Partitioning test and training set
X_train, X_test, y_train, y_test = train_test_split(df['review'].values.astype('U'),
                                                    df['label'].values.astype('U'),
                                                    test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(24496,)
(6125,)


Acquiring Count Vectorizer from the train set.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(X_train)
X_train_counts.shape

(24496, 70747)

Unique and effective words are more important. Therefore, we need to use TF*IDF Vectorization

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(24496, 70747)

Train Naive Bayes classifier

In [15]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train_tfidf, y_train)
X_test_counts = count_vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

Model is trained, it's time to test it.

In [22]:
# Testing with test set and classifier.
y_pred = classifier.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:10], y_pred[:]):
    print('%r => .%s' % (review, sentiment))

'hızlı kargo teşekkürler ertesi gün ürün elimize ulaştı çalışanlar ilgili tavsiye ederim' => .pos
'ürün kaliteli kullanışlı sadece kablo uzunluk seçeneği olsaydı güzel olurduhızlı kargo teşekkürler' => .pos
'diorun parfümleri kalıcılıkla başarılılar parfüm yoğun hafif olmayan bir koku genç birinde fazla durur' => .neg
'arkadaslar urunu cumartesi siparis verdim sali gunu elime ulasti gayet hizli geldi abimle ikimiz 2 saatte kurabildik hic bir eksik parca yoktu gorunumu gorseldeki gibiydi uretici firmaya hepsiburadacom a tesekkurler' => .pos
'pillerle uğraşmaktan bıkanlar evde bulunması gereken bir ürün10 led an adınlatma 3 led üst adınlatma kısa sürede şarj süresi var' => .pos
'doğuyu özellikle iranın tarihsel gelişimi başarılı tasvir edilmiş sıkılmadan okudum yapı kredi yayınlarını zaten beğenirim kitap diğerleriyle aynı çizgidetavsiye ederim' => .pos
'kitaplara olan saygımdan dolayı zorlanarak ta olsa bitirdiğim kitap okumaktan soğutan bir kitap' => .pos
'tavan boyama niyetiyle aldığı

Performance results?

In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8651428571428571
